In [2]:
import numpy as np
import pandas as pd
import tensorflow
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing
import gensim
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
import nltk

In [3]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    stops = stopwords.words('english')
    #print(stops)
    porter = PorterStemmer()
    for word in sentence.split():
        if word in stops:
            sentence = sentence.replace(word, '')
        sentence = sentence.replace(word, porter.stem(word))
    return sentence.lower()

In [4]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('../content/drive/MyDrive/Reviews.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:

df = df[['Text','Score']]
df['Score'].isnull().sum()
df['Score'].isnull().sum()
df.drop_duplicates(subset=['Text','Score'],keep='first',inplace=True)

def set_sent(score):
    if score<=2:
        return 0
    elif score==3:
        return 1
    else:
        return 2

df['sentiment']=df['Score'].apply(set_sent)
df = df[['Text','sentiment']]

# Separate into three sentiment groups
df_neg1 = df[df['sentiment']==0]
df_0 = df[df['sentiment']==1]
df_1 = df[df['sentiment']==2]

n = 10000

df_neg1 = df_neg1.sample(n=n, random_state=42, replace=False)
df_0 = df_0.sample(n=n, random_state=42, replace=False)
df_1 = df_1.sample(n=n, random_state=42, replace=False)
# print("df_neg1: ", df_neg1)
# print("df_0: ", df_0)
# print("df_1: ", df_1)

sub_df = pd.concat([df_neg1, df_0, df_1], axis=0)
X = sub_df['Text']

y = sub_df['sentiment']

In [6]:
mes = []
for i in X:
    mes.append(i.split())
print(mes[:2])

[['Vile', 'and', 'revolting', '"tahini";', 'absolutely', 'ruined', 'a', 'batch', 'a', 'hummus.', 'This', 'tahini', 'has', 'a', 'bitter,', 'metallic', 'and', 'chemical', 'like', 'taste.', 'It', 'is', 'not', 'a', 'matter', 'of', 'not', 'knowing', 'how', 'to', 'use', 'it', '(it', 'separates,', 'mix', 'it', 'up', 'just', 'like', 'natural', 'peanut', 'butter)', 'it', 'is', 'a', 'matter', 'of', 'an', 'awful', 'product.', 'Stay', 'far,', 'far', 'away', 'and', 'ignore', 'the', '5', 'star', 'reviews', 'which', 'sound', 'suspiciously', 'like', 'they', 'were', 'written', 'by', 'Joyva.', 'There', 'is', 'far', 'better', 'tahini', 'out', 'there.'], ['They', 'will', 'send', 'you', '70%', 'cocoa', 'lindt', 'chocolate', 'but', 'it', 'has', 'a', '"new', 'recipe".', "It's", '30', 'calories', 'more', 'a', 'serving,', 'all', 'sugar,', 'and', 'takes', 'like', 'crappy', 'milk', 'chocolate', "you'd", 'get', 'in', 'a', 'cheap', 'easter', 'bunny.', 'Has', 'none', 'of', 'the', 'cocoa', 'bite', 'that', 'it', 'use

In [7]:
w2v_model = Word2Vec(mes, window=10, min_count=1, workers=16)
print(w2v_model)

Word2Vec<vocab=120442, vector_size=100, alpha=0.025>


In [8]:
# Preprocess the text data
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2)

X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

max_length = 100

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
vocab_size = len(tokenizer.word_index) + 1
# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50
maxlen = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/100
375/375 [==============================] - 51s 127ms/step - loss: -27866.0215 - accuracy: 0.3531 - val_loss: -278851.0000 - val_accuracy: 0.3872
Epoch 2/100
375/375 [==============================] - 17s 46ms/step - loss: -10812046.0000 - accuracy: 0.3959 - val_loss: -44406216.0000 - val_accuracy: 0.4002
Epoch 3/100
375/375 [==============================] - 10s 28ms/step - loss: -230854320.0000 - accuracy: 0.4050 - val_loss: -596000576.0000 - val_accuracy: 0.3988
Epoch 4/100
375/375 [==============================] - 5s 14ms/step - loss: -1609089536.0000 - accuracy: 0.4068 - val_loss: -3237339648.0000 - val_accuracy: 0.4007
Epoch 5/100
375/375 [==============================] - 4s 12ms/step - loss: -6647575552.0000 - accuracy: 0.4106 - val_loss: -11516644352.0000 - val_accuracy: 0.4008
Epoch 6/100
375/375 [==============================] - 4s 12ms/step - loss: -19912839168.0000 - accuracy: 0.4126 - val_loss: -30719424512.0000 - val_accuracy: 0.4018
Epoch 7/100
375/375 [===